In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'../')

from common.featuresutils import load_features
from common.flagutils import load_flag_list
from common.flagutils import load_o3_flags
from common.flagutils import build_config
from common.flagutils import get_cmd_string_from_config

In [ ]:
data = pd.read_csv('CE.results.csv')
data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
data.loc[:, 'Benchmark'] = data['Benchmark'].str.replace('.x', '')

average_data = data.groupby(['Benchmark','Flags', 'Type', 'RunId'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
average_data.head()

In [ ]:
all_flags = load_flag_list()
print(len(all_flags))

In [ ]:
def best_configuration(variable, benchmark, average_data):
    benchmark_data = average_data.loc[average_data["Benchmark"] == benchmark]
    min_index = benchmark_data[variable].idxmin()
    return benchmark_data.loc[min_index]["Flags"]


def get_o3_config():
    return get_cmd_string_from_config(build_config(all_flags, load_o3_flags(), '-O3'))

In [ ]:
benchmarks = data["Benchmark"].unique()
print(benchmarks)

In [ ]:
X = load_features([b.split('.')[0] for b in benchmarks])
Y = []

for benchmark in benchmarks:
    config_str = best_configuration('Energy', benchmark, average_data)
    if config_str == '-O3':
        config_str = get_o3_config()
    
    labels = []
    
    config = config_str[4:].split(' ')
    for i, flag in enumerate(config):
        if flag == all_flags[i]:
            labels.append(1) # Flag is turned on
        elif flag == '-fno-' + all_flags[i][2:]:
            labels.append(0) # FLag is turned off -fno
        else:
            print("ERROR:" + flag)
    
    Y.append(labels)

Y = np.array(Y)
print(X.shape)
print(Y.shape)